In [ ]:
import numpy as np
import pandas as pd
from bokeh.io import output_notebook, show
output_notebook()

In [ ]:
from app.utils.process_gtimelog import get_work_df
raw = get_work_df()
raw.head()

In [ ]:
import datetime
today = datetime.date(2015, 7, 13)
just_today = raw[(raw.timestamp.dt.date == today)]
just_today = just_today[just_today.activity != 'start']

# Clean & add some useful columns
just_today.activity = just_today.activity.str.capitalize()
just_today['human'] = just_today.delta.dt.seconds / (60 * 60)
just_today.human = just_today.human.round(2)
just_today['parent_activity'] = just_today.activity.str.split(' - ').str[0]
just_today['sub_activity'] = just_today.activity.str.split(' - ').str[1]
just_today.sub_activity = np.where(just_today.activity == just_today.parent_activity, 'general', just_today.sub_activity)
just_today.sub_activity = just_today.sub_activity.str.capitalize()

just_today.tail()

In [ ]:
# Categorize (probably not necessary here)
just_today.parent_activity = just_today.parent_activity.astype('category')
just_today.parent_activity.cat.categories

In [ ]:
from bokeh.charts import Bar
from bokeh.io import hplot, vplot
from bokeh.models import Range1d, DataTable, ColumnDataSource, TableColumn, Paragraph
from bokeh.palettes import Spectral4

def make_bar(category, data):
    bar = Bar(
        data, 
        width=200, height=200, 
        palette=[Spectral4[i], '#dddddd'], 
        stacked=True)
    bar.toolbar_location = None
    bar.outline_line_color = None
    bar.y_range = Range1d(0, 8)
    bar.min_border = 5
    bar.min_border_top = 10
    return bar

def make_table(category, data):
    totalled = pd.concat(
        [
            data, 
            pd.DataFrame({'human': data.human.sum()}, index=['%s - Total' % category])
        ]
    )
    source = ColumnDataSource(totalled)
    table = DataTable(
        source=source,
        columns=[
            TableColumn(field="sub_activity", title=category),
            TableColumn(field="human", title="Total")
        ],
        width=300
    )
    return table

for i, category in enumerate(just_today.parent_activity.cat.categories):
    parent_df = just_today[just_today.parent_activity == category]
    summed = parent_df.groupby('sub_activity').sum().sort('human', ascending=False)
    summed['from total'] = summed.human.sum() - summed.human
    bar = make_bar(category, summed)
    table = make_table(category, summed)
    show(vplot(hplot(bar, table)))